## Modify Data accrording previous conclusions

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import sys
sys.path.insert(0, 'G:/work/GitHub/ml_baseline_things/functions/')
import supportFunctions as sf
import featureEngineringFunctions as fef

dirPath = '2-featuresPack/'


## Conclusions from previous notebook:

* drop features:
 * passengerId
 * cabin (after extract the deck)
 * ticket
 * embarked
* missing data to recover:
 * <s>age</s>
* convert categorial to:
 * <s>sex - to int and to several columns</s>
* new features from:
 * <s>cabin (extract decks)</s>
 * <s>name (extract title)</s>
 * <s>age bands</s>
 * <s>fare </s>normalize and (?)<s> make bands</s>
 * <s>sibsp & parch - may be generated isAlone hasChildren/Parents hasHasb/Wife/</s>
 
 Finally convert to smaller memory size

In [2]:
#dataRawTrain, dataRawTest, _ = sf.load_raw_data()
dataTrain, dataTest, _ = sf.load_raw_data()
dataAll = dataTrain.append(dataTest, ignore_index=True)
#dataAll.tail()

Train (891, 12); Test (418, 11); Total (1309, 12).


In [3]:
trainVStest = len(dataTrain)

In [4]:
dataAll.isnull().sum()

Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64

## Id + Survived

In [5]:
# save base attrs
fef.save_to_file(dataAll['PassengerId'], 'PassengerId', dirPath)
fef.save_to_file(dataAll['Survived'], 'Survived', dirPath)

PassengerId saved.
Survived saved.


0

### Class

In [6]:
fef.save_to_file (dataAll['Pclass'], 'Pclass_LE', dirPath)

Pclass_LE saved.


0

### SEX

In [7]:
#no missing values

# encode SEX feature

#save as OHE
featureSex, col_ = fef.fe_cat_ohe(dataAll, 'Sex')
fef.save_to_file (featureSex, 'Sex_OHE', dirPath)

#save as LE
dataAll['Sex'] = dataAll['Sex'].map({'female': 1, 'male': 0}).astype(int)
featureSex, col_ = fef.fe_cat_le(dataAll, 'Sex')

fef.save_to_file (featureSex, 'Sex_LE', dirPath)

dataAll.head()

Sex_OHE saved.
Sex_LE saved.


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,0,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,1,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,1,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,0,0,0.0,373450


### Extract title from name

In [8]:
# extract title from name
dataAll['Title'] = dataAll['Name'].str.extract('([A-Za-z]+)\.', expand=True)
    
dataAll.Title.value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Major         2
Mlle          2
Ms            2
Sir           1
Don           1
Lady          1
Jonkheer      1
Dona          1
Capt          1
Countess      1
Mme           1
Name: Title, dtype: int64

In [9]:
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr', 'Don': 'Mr', 'Mme': 'Miss',
          'Jonkheer': 'Mr', 'Lady': 'Mrs', 'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}

dataAll.replace({'Title': mapping}, inplace=True)
dataAll.Title.value_counts()    

Mr        767
Miss      265
Mrs       200
Master     61
Dr          8
Rev         8
Name: Title, dtype: int64

In [10]:
dataAll[:trainVStest][['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Dr,0.428571
1,Master,0.575000
2,Miss,0.704301
3,Mr,0.160000
4,Mrs,0.795276
5,Rev,0.000000


In [11]:
#save as OHE
featureTitle, col_ = fef.fe_cat_ohe(dataAll, 'Title')
fef.save_to_file (featureTitle, 'Title_OHE', dirPath)

#save as LE
featureTitle, col_ = fef.fe_cat_le(dataAll, 'Title')
fef.save_to_file (dataAll['Title'], 'Title_LE', dirPath)

Title_OHE saved.
Title_LE saved.


0

### Missing data recovery, AGE
Strategy = median Age for Pclass=1 and Gender=0, Pclass=1 and Gender=1, and so on...

In [12]:
sf.check_missing_data(dataAll[:trainVStest], dataAll[trainVStest:], ['Age'])

Train:
 Age    177
dtype: int64 

Test:
 Age    86
dtype: int64


0

In [13]:
# fill the missing value for Age column with median of its title
titles = list(dataAll.Title.unique())
for title in titles:
    age = dataAll.groupby('Title')['Age'].median().loc[title]
    dataAll.loc[(dataAll.Age.isnull()) & (dataAll.Title == title),'Age'] = age

In [14]:
sf.check_missing_data(dataAll[:trainVStest], dataAll[trainVStest:], ['Age'])

Train:
 Age    0
dtype: int64 

Test:
 Age    0
dtype: int64


0

In [15]:
fef.save_to_file (dataAll['Age'], 'Age_cont', dirPath)

Age_cont saved.


0

### Extract AgeBands from Age

In [16]:
dataAll['AgeBin'] = pd.qcut(dataAll['Age'], 4)
dataAll[:trainVStest][['AgeBin', 'Survived']].groupby(['AgeBin'], as_index=False).mean().sort_values(by='AgeBin', ascending=True)

,AgeBin,Survived
0,"(0.169, 22.0]",0.450185
1,"(22.0, 30.0]",0.276094
2,"(30.0, 36.0]",0.519685
3,"(36.0, 80.0]",0.367347


In [17]:
featureAgeBin, col_ = fef.fe_cat_le(dataAll, 'AgeBin')
fef.save_to_file (featureAgeBin, 'AgeBin_LE', dirPath)
dataAll.drop('AgeBin', axis=1, inplace=True)

AgeBin_LE saved.


In [18]:
#dataAll.head(5)


### sibsp & parch new features generation

In [19]:
dataAll['FamilySize_LE'] = dataAll['SibSp'] + dataAll['Parch'] + 1
fef.save_to_file (dataAll['FamilySize_LE'], 'FamilySize_LE', dirPath)

fef.save_to_file (dataAll['SibSp'], 'SibSp_LE', dirPath)
fef.save_to_file (dataAll['Parch'], 'Parch_LE', dirPath)

dataAll[:trainVStest][['FamilySize_LE', 'Survived']].groupby(['FamilySize_LE'], as_index=False).mean().sort_values(by='Survived', ascending=False)

FamilySize_LE saved.
SibSp_LE saved.
Parch_LE saved.


,FamilySize_LE,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [20]:
dataAll['IsAlone_LE'] = 0
dataAll.loc[dataAll['FamilySize_LE'] == 1, 'IsAlone_LE'] = 1
dataAll['HasWifeHasb_LE'] = 0
dataAll.loc[dataAll['SibSp'] > 0 , 'HasWifeHasb_LE'] = 1
dataAll['HasBloodRelatives_LE'] = 0
dataAll.loc[dataAll['Parch'] > 0 , 'HasWifeHasb_LE'] = 1

fef.save_to_file (dataAll['IsAlone_LE'], 'IsAlone_LE', dirPath)
fef.save_to_file (dataAll['HasWifeHasb_LE'], 'HasWifeHasb_LE', dirPath)
fef.save_to_file (dataAll['HasBloodRelatives_LE'], 'HasBloodRelatives_LE', dirPath)

dataAll[:trainVStest][['IsAlone_LE', 'Survived']].groupby(['IsAlone_LE'], as_index=False).mean()

IsAlone_LE saved.
HasWifeHasb_LE saved.
HasBloodRelatives_LE saved.


,IsAlone_LE,Survived
0,0,0.505650
1,1,0.303538


In [21]:
#dataTrain

### Syntetical union of features

In [22]:
dataAll['Age*Class'] = (dataAll.Age * dataAll.Pclass).astype(int)

fef.save_to_file (dataAll['Age*Class'], 'Age-Class_cont', dirPath)
    
dataAll.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

Age-Class_cont saved.


,Age*Class,Age,Pclass
0,66,22.0,3
1,38,38.0,1
2,78,26.0,3
3,35,35.0,1
4,105,35.0,3
5,90,30.0,3
6,54,54.0,1
7,6,2.0,3
8,81,27.0,3
9,28,14.0,2


### Fare levels

In [23]:
dataAll.isnull().sum()

Age                        0
Cabin                   1014
Embarked                   2
Fare                       1
Name                       0
Parch                      0
PassengerId                0
Pclass                     0
Sex                        0
SibSp                      0
Survived                 418
Ticket                     0
Title                      0
FamilySize_LE              0
IsAlone_LE                 0
HasWifeHasb_LE             0
HasBloodRelatives_LE       0
Age*Class                  0
dtype: int64

In [24]:
#for test only
def get_median_fare_by_item (pClass, i):
    return dataAll.query('Pclass == {}'.format(pClass))['Fare'].mean()
    
for i in range(len (dataAll)):
    if pd.isnull(dataAll.loc[i,'Fare']):
        dataAll.loc[i,'Fare'] = get_median_fare_by_item (dataAll.loc[i,'Pclass'], i)

In [25]:
dataAll['FareBin'] = pd.qcut(dataAll['Fare'], 5)
dataAll[['FareBin', 'Survived']].groupby(['FareBin'], as_index=False).mean().sort_values(by='FareBin', ascending=True)

,FareBin,Survived
0,"(-0.001, 7.854]",0.217877
1,"(7.854, 10.5]",0.201087
2,"(10.5, 21.558]",0.426901
3,"(21.558, 41.579]",0.443243
4,"(41.579, 512.329]",0.645349


In [26]:
#for dataset in [dataTrain, dataTest]:
#    dataset['Fare_lev'] = 0
#    dataset['Fare_lev'].loc[dataset['Fare'] <= 7.85] = 0
#    dataset['Fare_lev'].loc[(dataset['Fare'] > 7.85) & (dataset['Fare'] <= 10.5)] = 1
#    dataset['Fare_lev'].loc[(dataset['Fare'] > 10.5) & (dataset['Fare'] <= 21.67)] = 2
#    dataset['Fare_lev'].loc[(dataset['Fare'] > 21.67) & (dataset['Fare'] <= 39.7)] = 3
#    dataset['Fare_lev'].loc[dataset['Fare'] > 39.7] = 4

#fef.save_to_file (dataTrain['Fare_lev'], dataTest['Fare_lev'], dirPath, 'Fare_lev') 

#dataTrain = dataTrain.drop(['FareBin'], axis=1)

#fef.save_to_file (dataTrain['Fare'], dataTest['Fare'], dirPath, 'Fare_cnt')

#for dataset in [dataTrain, dataTest]:
#    dataset.drop(['Fare'], axis=1)

fef.save_to_file (dataAll['Fare'], 'Fare_cont', dirPath)

featureAgeBin, col_ = fef.fe_cat_le(dataAll, 'FareBin')
fef.save_to_file (featureAgeBin, 'FareBin_LE', dirPath)
dataAll.drop('FareBin', axis=1, inplace=True)




Fare_cont saved.
FareBin_LE saved.


### Cabin to decks

In [27]:
#decks = {"U": 0, "A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "T": 0}

dataAll['Deck'] = 0
dataAll['Deck'] = dataAll['Cabin'].fillna('U').apply(lambda c: c[0])
print (dataAll.Deck.value_counts())

U    1014
C      94
B      65
D      46
E      41
A      22
F      21
G       5
T       1
Name: Deck, dtype: int64


In [28]:
dataAll['Deck'] = dataAll['Deck'].replace('T', 'U')

print (dataAll.Deck.value_counts())

U    1015
C      94
B      65
D      46
E      41
A      22
F      21
G       5
Name: Deck, dtype: int64


In [29]:
#save as OHE
featureDeck, col_ = fef.fe_cat_ohe(dataAll, 'Deck')
fef.save_to_file (featureDeck, 'Deck_OHE', dirPath)

#save as LE
featureDeck, col_ = fef.fe_cat_le(dataAll, 'Deck')
fef.save_to_file (featureDeck, 'Deck_LE', dirPath)

Deck_OHE saved.
Deck_LE saved.


0

## Stolen features

### FamilySurvival

In [30]:
# This feature is from S.Xu, https://www.kaggle.com/shunjiangxu/blood-is-thicker-than-water-friendship-forever
dataAll['LastName'] = dataAll['Name'].apply(lambda x: str.split(x, ",")[0])
#total['Fare'].fillna(total['Fare'].mean(), inplace=True)

default_survival_rate = 0.5
dataAll['FamilySurvival'] = default_survival_rate

for grp, grp_df in dataAll[['Survived','Name', 'LastName', 'Fare', 'Ticket', 'PassengerId',
                           'SibSp', 'Parch', 'Age', 'Cabin']].groupby(['LastName', 'Fare']):
    
    if (len(grp_df) != 1):
        # A Family group is found.
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                dataAll.loc[dataAll['PassengerId'] == passID, 'FamilySurvival'] = 1
            elif (smin==0.0):
                dataAll.loc[dataAll['PassengerId'] == passID, 'FamilySurvival'] = 0

print("Number of passengers with family survival information:", 
      dataAll.loc[dataAll['FamilySurvival']!=0.5].shape[0])
dataAll['FamilySurvival'].value_counts()


Number of passengers with family survival information: 420


0.5    889
1.0    218
0.0    202
Name: FamilySurvival, dtype: int64

In [31]:
for _, grp_df in dataAll.groupby('Ticket'):
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            if (row['FamilySurvival'] == 0) | (row['FamilySurvival']== 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    dataAll.loc[dataAll['PassengerId'] == passID, 'FamilySurvival'] = 1
                elif (smin==0.0):
                    dataAll.loc[dataAll['PassengerId'] == passID, 'FamilySurvival'] = 0
                        
print("Number of passenger with family/group survival information: " 
      +str(dataAll[dataAll['FamilySurvival']!=0.5].shape[0]))
fef.save_to_file (dataAll['FamilySurvival'], 'FamilySurvival_bin', dirPath)
dataAll['FamilySurvival'].value_counts()

Number of passenger with family/group survival information: 546
FamilySurvival_bin saved.


0.5    763
1.0    303
0.0    243
Name: FamilySurvival, dtype: int64

In [32]:
#dataAll.head()

### Drop features

In [33]:
#no sense in drop - all features are in files

### Final checkers missing values

In [34]:
dataAll.isnull().sum()
#TODO неплохо было бы проверять на уникальные значения в тесте и трейне - чтобы бились

Age                        0
Cabin                   1014
Embarked                   2
Fare                       0
Name                       0
Parch                      0
PassengerId                0
Pclass                     0
Sex                        0
SibSp                      0
Survived                 418
Ticket                     0
Title                      0
FamilySize_LE              0
IsAlone_LE                 0
HasWifeHasb_LE             0
HasBloodRelatives_LE       0
Age*Class                  0
Deck                       0
LastName                   0
FamilySurvival             0
dtype: int64

## what can be done more

In [35]:
- можно восстановить палубу исходя из класса и ценника билета
1 класс - A-E, самые дорогие в B
2 и 3 класс - D-G, в D - лучшие

SyntaxError: invalid syntax (<ipython-input-35-034a47c91068>, line 1)

In [ ]:
#features_from_other = ['Survived','Pclass','Sex','Family_Size','Family_Survival','Fare_Bin','Age_Bin']
featureListForTest = [
#'PassengerId',
#'Survived',
'AGE_cont',
'AgeBin_LE',
'Age-Class_cont',
'Deck_LE',
'Deck_OHE',
'FamilySize_LE',
'Fare_cont',
'FareBin_LE',
'HasBloodRelatives_LE',
'HasWifeHasb_LE',
'IsAlone_LE',
'Parch_LE',
'Pclass_LE',
'Sex_LE',
'Sex_OHE',
'SibSp_LE',
'Title_LE',
'Title_OHE',
'FamilySurvival_bin'
]
dataAlltmp = fef.create_dataset_from_features (featureListForTest, dirPath)
dataAlltmp#.head()